In [457]:
from collections import deque
import gym
import matplotlib.pyplot as plt
from IPython import display
import time
import torch
import torch.nn as nn
import numpy as np
import random

In [458]:
# env = gym.make('MountainCar-v0')
# state = env.reset()
# for i in range(1):
# #     plt.imshow(env.render(mode='rgb_array'))
# #     display.display(plt.gcf())
# #     display.clear_output(wait=True)

#     env.render()
#     action = env.action_space.sample()
#     state,reward,done,info = env.step(0)
# #     print(state.shape)
# #     print(f'{done}/action={action}')
#     print(reward)
#     time.sleep(0.02)
#     if done:
#         print()
#         break
# time.sleep(5)
# env.close()

In [459]:
env = gym.make('MountainCar-v0')
state = env.reset()
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
batch_size = 64
epochs = 1000
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [460]:
class NNet(nn.Module):
    def __init__(self):
        super(NNet,self).__init__()
        self.lin1 = nn.Linear(state_size,32)
        self.lin2 = nn.Linear(32,64)
        self.lin3 = nn.Linear(64,32)
        self.lin4 = nn.Linear(32,action_size)
        self.relu = nn.ReLU()
    def forward(self,x):
#         pred = torch.from_numpy(x.reshape(1,state_size)).float().to(device)
        pred = self.lin1(x)
        pred = self.relu(pred)
        pred = self.lin2(pred)
        pred = self.relu(pred)
        pred = self.lin3(pred)
        pred = self.relu(pred)
        pred = self.lin4(pred)
        return pred

In [461]:
# # a = NNet()
# # b = NNet()
# x = torch.tensor([[1,2]],dtype=torch.float32)
# b.load_state_dict(a.state_dict())
# print(a(x))
# print(b(x))
env.close()

In [462]:
class dqn_agent():
    def __init__(self,state_size,action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=1200)
        self.gamma = 0.95
        self.epsilon = 1
        self.epsilon_rate = 0.995
        self.epsilon_lim = 0.01
        self.lr = 0.001
        self.model = NNet().to(device)
        self.target_model = NNet().to(device)
        self.criterion = nn.MSELoss()
        self.optim = torch.optim.Adam(self.model.parameters(),lr=self.lr)
    def train(self,batch_size):
        batch = random.sample(self.memory,batch_size)
        self.target_model.load_state_dict(self.model.state_dict())
        for state,action,reward,next_state,done in batch:
            state = torch.from_numpy(state.reshape(1,state_size)).float().to(device)
            next_state = torch.from_numpy(next_state.reshape(1,state_size)).float().to(device)
            pred = self.model(state)[0][action]
            target = torch.tensor(reward,dtype=torch.float32).to(device)
            if not done:
                action_index = torch.argmax(self.model(next_state)[0])
                target = target+self.target_model(next_state)[0][action_index]
            loss = self.criterion(pred,target.to(device))
            loss.backward()
            self.optim.step()
            self.optim.zero_grad()
        if self.epsilon>self.epsilon_lim:
            self.epsilon*=self.epsilon_rate
    def action(self,state):
        if np.random.rand()<=self.epsilon:
            return torch.tensor(random.randrange(self.action_size))
        else:
            state = torch.from_numpy(state.reshape(1,state_size)).float().to(device)
            return torch.argmax(self.model(state)[0])
    def remember(self,state,action,reward,next_state,done):
        self.memory.append((state,action,reward,next_state,done))
    def count_reward(self,state,next_state,action):
        pass
        

In [463]:
agent = dqn_agent(state_size,action_size)

for epoch in range(epochs):
    state = env.reset()
    
    done = False
    times = 0
    while not done:
        action = agent.action(state)
        if epoch%20 == 0:
            env.render()
#         if epoch%20 == 0:
#             time.sleep(0.02)
        next_state, reward, done, info = env.step(action.to('cpu').numpy())
        if done:
            if next_state[0] >=0.5:
                reward = 10
            else:
                reward = -10
        else:
            if next_state[0] > -0.4:
                reward = (1+state[0])**2
#         else:
#             if next_state[0]>=0.35:
#                 reward += 1+(next_state[0]-0.35)*10
#         else:
#             if state[1] >= 0 and state[0]>=-0.5:
#                 if action==0:
#                     reward = -1
#                 elif action==1:
#                     reward = -0.8
#                 else:
#                     reward = -0.1
                    
#             elif state[1]<0 and state[0]>=-0.5:
#                 if action==0:
#                     reward = -0.1
#                 elif action==1:
#                     reward = -0.8
#                 else:
#                     reward = -1
            
#             elif state[1]>=0 and state[0]<-0.5:
#                 if action==0:
#                     reward = -1
#                 elif action==1:
#                     reward = -0.8
#                 else:
#                     reward = -0.1
#             else:
#                 if action==0:
#                     reward = -0.1
#                 elif action==1:
#                     reward = -0.8
#                 else:
#                     reward = -1
            
                
        
        agent.remember(state,action,reward,next_state,done)
#         if reward==1000:
#             for i in range(99):
#                 agent.remember(state,action,reward,next_state,done)
        state = next_state
        if done:
            print(f'epoch = {epoch}/{epochs}, score = {times}, epsilon = {agent.epsilon:.4f}')
        times+=1
        if (times>120 or epoch <200) and len(agent.memory)>=batch_size and times%16==0:
            agent.train(batch_size)
    if epoch%20 == 0:
        env.close()
    if times <=110 and epoch>200:
        break

epoch = 0/1000, score = 199, epsilon = 0.9559
epoch = 1/1000, score = 199, epsilon = 0.9001
epoch = 2/1000, score = 199, epsilon = 0.8475
epoch = 3/1000, score = 199, epsilon = 0.7981
epoch = 4/1000, score = 199, epsilon = 0.7515
epoch = 5/1000, score = 199, epsilon = 0.7076
epoch = 6/1000, score = 199, epsilon = 0.6663
epoch = 7/1000, score = 199, epsilon = 0.6274
epoch = 8/1000, score = 199, epsilon = 0.5908
epoch = 9/1000, score = 199, epsilon = 0.5563
epoch = 10/1000, score = 199, epsilon = 0.5238
epoch = 11/1000, score = 199, epsilon = 0.4932
epoch = 12/1000, score = 199, epsilon = 0.4644
epoch = 13/1000, score = 199, epsilon = 0.4373
epoch = 14/1000, score = 199, epsilon = 0.4118
epoch = 15/1000, score = 199, epsilon = 0.3878
epoch = 16/1000, score = 199, epsilon = 0.3651
epoch = 17/1000, score = 199, epsilon = 0.3438
epoch = 18/1000, score = 199, epsilon = 0.3237
epoch = 19/1000, score = 199, epsilon = 0.3048
epoch = 20/1000, score = 199, epsilon = 0.2870
epoch = 21/1000, score 

epoch = 173/1000, score = 199, epsilon = 0.0100
epoch = 174/1000, score = 199, epsilon = 0.0100
epoch = 175/1000, score = 199, epsilon = 0.0100
epoch = 176/1000, score = 199, epsilon = 0.0100
epoch = 177/1000, score = 199, epsilon = 0.0100
epoch = 178/1000, score = 199, epsilon = 0.0100
epoch = 179/1000, score = 199, epsilon = 0.0100
epoch = 180/1000, score = 199, epsilon = 0.0100
epoch = 181/1000, score = 199, epsilon = 0.0100
epoch = 182/1000, score = 199, epsilon = 0.0100
epoch = 183/1000, score = 199, epsilon = 0.0100
epoch = 184/1000, score = 199, epsilon = 0.0100
epoch = 185/1000, score = 199, epsilon = 0.0100
epoch = 186/1000, score = 192, epsilon = 0.0100
epoch = 187/1000, score = 164, epsilon = 0.0100
epoch = 188/1000, score = 126, epsilon = 0.0100
epoch = 189/1000, score = 116, epsilon = 0.0100
epoch = 190/1000, score = 122, epsilon = 0.0100
epoch = 191/1000, score = 199, epsilon = 0.0100
epoch = 192/1000, score = 126, epsilon = 0.0100
epoch = 193/1000, score = 199, epsilon =

In [466]:
# play test:
for epoch in range(100):
    state = env.reset()   
    done = False
    times = 0
    while not done:
        action = agent.action(state)
        env.render()
        next_state, reward, done, info = env.step(action.to('cpu').numpy())
        state = next_state
        if done:
            print(f'episode = {epoch}/{100}, score = {1/times*19900 if times != 199 else 0:.2f}')
        times+=1
    env.close()
        

episode = 0/100, score = 120.61
episode = 1/100, score = 234.12
episode = 2/100, score = 203.06
episode = 3/100, score = 193.20
episode = 4/100, score = 226.14
episode = 5/100, score = 236.90
episode = 6/100, score = 193.20
episode = 7/100, score = 120.61
episode = 8/100, score = 120.61
episode = 9/100, score = 218.68
episode = 10/100, score = 121.34
episode = 11/100, score = 115.70
episode = 12/100, score = 201.01
episode = 13/100, score = 114.37
episode = 14/100, score = 115.70
episode = 15/100, score = 120.61
episode = 16/100, score = 119.88
episode = 17/100, score = 117.06
episode = 18/100, score = 228.74
episode = 19/100, score = 185.98
episode = 20/100, score = 231.40
episode = 21/100, score = 119.16
episode = 22/100, score = 115.70
episode = 23/100, score = 119.88
episode = 24/100, score = 228.74
episode = 25/100, score = 122.09
episode = 26/100, score = 119.16
episode = 27/100, score = 119.16
episode = 28/100, score = 226.14
episode = 29/100, score = 226.14
episode = 30/100, sc

In [465]:
# save good model
file = "mountain_car_gpu_model_best.path"
torch.save(agent.model.state_dict(),file)